# Imports

In [1]:
import importlib
from functools import partial

from torch import nn
import torch
import pandas as pd
import numpy as np

import decision_learning.modeling.pipeline
import decision_learning.benchmarks.shortest_path_grid.data

from decision_learning.modeling.models import LinearRegression
from decision_learning.modeling.pipeline import lossfn_experiment_pipeline, lossfn_hyperparam_grid
from decision_learning.benchmarks.shortest_path_grid.data import genData

In [2]:
import decision_learning.modeling.train
importlib.reload(decision_learning.modeling.train)

importlib.reload(decision_learning.modeling.pipeline)
from decision_learning.modeling.pipeline import lossfn_experiment_pipeline, lossfn_hyperparam_grid 

import decision_learning.benchmarks.shortest_path_grid.data
importlib.reload(decision_learning.benchmarks.shortest_path_grid.data)
from decision_learning.benchmarks.shortest_path_grid.data import genData

import decision_learning.modeling.loss
importlib.reload(decision_learning.modeling.loss)
import decision_learning.modeling.loss

# Example Setup

### Optimization Model

# Tell people what the optimization model output should look like

In [3]:
from decision_learning.benchmarks.shortest_path_grid.oracle import opt_oracle as shortest_path_solver


### Data Generation Setup

In [4]:
torch.manual_seed(105)
indices_arr = torch.randperm(100000)
indices_arr_test = torch.randperm(100000)

sim = 0
n_arr = [200, 400, 800, 1600]
ep_arr = ['unif', 'normal']
trials = 100

exp_arr = []
for n in n_arr:
    for ep in ep_arr:
        for t in range(trials):
            exp_arr.append([n, ep, t])

# setup
exp = exp_arr[0]
ep_type = exp[1]
trial = exp[2]

# generate data
grid = (5, 5)  # grid size
num_data = exp[0]  # number of training data
num_feat = 5  # size of feature
deg = 6  # polynomial degree
e = .3  # noise width

# path planting for shortest path example
planted_good_pwl_params = {'slope0':0, 
                    'int0':2,
                    'slope1':0, 
                    'int1':2}
planted_bad_pwl_params = {'slope0':4, 
                    'int0':0,
                    'slope1':0, 
                    'int1':2.2}

plant_edge = True

print(num_data, ep_type, trial)

200 unif 0


In [5]:
len(exp_arr)

800

# Testing Pipeline
Necessary components
- data (features, true costs): train-test splits
- prediction model
- optimization model
- existing loss functions (hyperparameter configs)
- user-defined loss functions
- misc params

In [6]:
importlib.reload(decision_learning.modeling.pipeline)
from decision_learning.modeling.pipeline import lossfn_experiment_pipeline, lossfn_hyperparam_grid 

importlib.reload(decision_learning.modeling.train)
from decision_learning.modeling.train import train

In [7]:
# ------------DATA------------
# training data
generated_data = genData(num_data=num_data+200,
        num_features=num_feat, 
        grid=grid, 
        deg=deg, 
        noise_type=ep_type,
        noise_width=e,
        seed=indices_arr[trial],     
        plant_edges=plant_edge,
        planted_good_pwl_params=planted_good_pwl_params,
        planted_bad_pwl_params=planted_bad_pwl_params)
# testing data
generated_data_test = genData(num_data=10000,
        num_features=num_feat, 
        grid=grid, 
        deg=deg, 
        noise_type=ep_type,
        noise_width=e,
        seed=indices_arr_test[trial],     
        plant_edges=plant_edge,
        planted_good_pwl_params=planted_good_pwl_params,
        planted_bad_pwl_params=planted_bad_pwl_params)

# ------------prediction model------------
pred_model = LinearRegression(input_dim=generated_data['feat'].shape[1],
                 output_dim=generated_data['cost'].shape[1])

# ------------optimization model------------
optmodel = partial(shortest_path_solver,size=5)

# ------------user-defined loss function------------
user_defined_loss_inputs = [{'loss_name':'cosine',
                      'loss':nn.CosineEmbeddingLoss,
                      'data': {'X': generated_data['feat'],
                               'true_cost':generated_data['cost'],
                               'input2':generated_data['cost'], 
                               'target':torch.ones(generated_data['cost'].shape[0])}
                      }
                     ]


### Get SPO, MSE, Cosine First

In [8]:
result_metrics, trained_models = lossfn_experiment_pipeline(X_train=generated_data['feat'],
                true_cost_train=generated_data['cost'],
                X_test=generated_data_test['feat'],
                true_cost_test=generated_data_test['cost_true'], 
                predmodel=pred_model,
                optmodel=optmodel,
                val_split_params={'test_size':200, 'random_state':42},                                                            
                loss_names=['SPO+', 'MSE', 'FYL', 'Cosine'],          
                user_defined_loss_inputs=user_defined_loss_inputs,                                                            
                training_configs={'num_epochs':100,
                                  'lr': 0.01,
                                 'dataloader_params': {'batch_size':200, 'shuffle':True}},
                save_models=True
                )

2024-12-10 16:37:24,189 - decision_learning.modeling.pipeline - INFO - Loss number 1/4, on loss function SPO+
2024-12-10 16:37:24,189 - decision_learning.modeling.pipeline - INFO - Trial 1/1 for running loss function SPO+, current hyperparameters: {}
2024-12-10 16:37:26,148 - decision_learning.modeling.pipeline - INFO - Loss number 2/4, on loss function MSE
2024-12-10 16:37:26,149 - decision_learning.modeling.pipeline - INFO - Trial 1/1 for running loss function MSE, current hyperparameters: {}
2024-12-10 16:37:27,275 - decision_learning.modeling.pipeline - INFO - Loss number 3/4, on loss function FYL
2024-12-10 16:37:27,275 - decision_learning.modeling.pipeline - INFO - Trial 1/1 for running loss function FYL, current hyperparameters: {}
2024-12-10 16:37:29,064 - decision_learning.modeling.pipeline - INFO - Loss number 4/4, on loss function Cosine
2024-12-10 16:37:29,065 - decision_learning.modeling.pipeline - INFO - Trial 1/1 for running loss function Cosine, current hyperparameters:

In [9]:
result_metrics[result_metrics.epoch == 99]

,epoch,train_loss,val_metric,test_regret,loss_name,hyperparameters
99,99,4.306511,0.211430,0.044372,SPO+,{}
199,99,2.063001,0.465148,0.236546,MSE,{}
299,99,4.205600,0.178077,0.028797,FYL,{}
399,99,0.030852,0.187353,0.072749,Cosine,{}
499,99,0.030852,0.178604,0.072749,cosine,None


### Input SPO+ as initialization into PG Loss

In [19]:
SPO_trained = trained_models['SPO+_{}']

In [20]:
PG_result_metrics, PG_trained_models = lossfn_experiment_pipeline(X_train=generated_data['feat'],
                true_cost_train=generated_data['cost'],
                X_test=generated_data_test['feat'],
                true_cost_test=generated_data_test['cost_true'], 
                predmodel=SPO_trained,
                optmodel=optmodel,
                val_split_params={'test_size':200, 'random_state':42},
                loss_names=['PG'],          
                loss_configs={'PG': {'h':[num_data**-.125, num_data**-.25, num_data**-.5, num_data**-1], 
                                     'finite_diff_type': ['B', 'C', 'F']
                                    }
                             },                
                training_configs={'num_epochs':100,
                                  'lr': 0.01,
                                 'dataloader_params': {'batch_size':200, 'shuffle':True}},
                save_models=False
                )

2024-12-10 15:54:08,024 - decision_learning.modeling.pipeline - INFO - Loss number 1/1, on loss function PG
2024-12-10 15:54:08,024 - decision_learning.modeling.pipeline - INFO - Trial 1/12 for running loss function PG, current hyperparameters: {'h': 0.5156692688606229, 'finite_diff_type': 'B'}
2024-12-10 15:54:09,272 - decision_learning.modeling.pipeline - INFO - Trial 2/12 for running loss function PG, current hyperparameters: {'h': 0.5156692688606229, 'finite_diff_type': 'C'}
2024-12-10 15:54:10,517 - decision_learning.modeling.pipeline - INFO - Trial 3/12 for running loss function PG, current hyperparameters: {'h': 0.5156692688606229, 'finite_diff_type': 'F'}
2024-12-10 15:54:11,765 - decision_learning.modeling.pipeline - INFO - Trial 4/12 for running loss function PG, current hyperparameters: {'h': 0.26591479484724945, 'finite_diff_type': 'B'}
2024-12-10 15:54:13,010 - decision_learning.modeling.pipeline - INFO - Trial 5/12 for running loss function PG, current hyperparameters: {'

## Combine Results

In [21]:
final_metrics = pd.concat([result_metrics, PG_result_metrics], ignore_index=True)

In [22]:
final_metrics.loc[final_metrics.groupby('loss_name')['val_metric'].idxmin()].sort_values(by='test_regret')

,epoch,train_loss,val_metric,test_regret,loss_name,hyperparameters
571,71,14.273172,0.156225,0.014403,PG,"{'h': 0.5156692688606229, 'finite_diff_type': ..."
297,97,4.174900,0.172062,0.029759,FYL,{}
353,53,0.030131,0.143116,0.045167,cosine,None
88,88,4.287334,0.195468,0.047049,SPO+,{}
195,95,2.152484,0.438158,0.241357,MSE,{}
